<a href="https://colab.research.google.com/github/leemnj/iterative-dna-reconstruction/blob/main/gen-seq-and-emb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sequence generation + embeddings (sequential model loading)

This notebook fetches gene sequences, generates sequences per model,
then creates cross-embeddings while loading one model at a time.

In [1]:
%%capture
# Colab setup
!git clone https://github.com/leemnj/iterative-dna-reconstruction.git
%cd iterative-dna-reconstruction
!pip -q install -e .
!pip -q install biopython # Install Biopython
!pip list | grep idr

from pathlib import Path
RESULTS_DIR = Path("data")  # 또는 "/content/drive/MyDrive/..."
RESULTS_DIR.mkdir(exist_ok=True)

fatal: destination path 'iterative-dna-reconstruction' already exists and is not an empty directory.
/content/iterative-dna-reconstruction
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for idr (pyproject.toml) ... done
idr                                      0.1.0               /content/iterative-dna-reconstruction


In [2]:
import gc
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import torch
# import sys

# sys.path.append('src') # Add src directory to Python path

from idr.utils import get_device, iter_models, load_model
from idr.preparation import (
    fetch_gene_sequences,
    sort_genes_by_length,
    DEFAULT_DECODING_STRATEGIES,
    get_sequence_from_ensembl,
    DEFAULT_GENES,
)

print('✅ 모든 모듈 임포트 완료')

✅ 모든 모듈 임포트 완료


In [ ]:
NCBI_EMAIL = 'dlalswo0321@gmail.com'
ITERATIONS = 30
MASK_RATIO = 0.2

RESULTS_DIR = Path('data')
SEQ_DIR = RESULTS_DIR / 'sequences'
EMB_DIR = RESULTS_DIR / 'embeddings'
SEQ_DIR.mkdir(parents=True, exist_ok=True)
EMB_DIR.mkdir(parents=True, exist_ok=True)

model_configs = {
    'DNABERT-2': 'zhihan1996/DNABERT-2-117M',
    'NT-v2-50m': 'InstaDeepAI/nucleotide-transformer-v2-50m-multi-species',
    'NT-v2-500m': 'InstaDeepAI/nucleotide-transformer-v2-500m-multi-species',
}
torch_dtype = 'float32'
model_labels = list(model_configs.keys())

print(f'NCBI email: {NCBI_EMAIL}')
print(f'ITERATIONS={ITERATIONS}, MASK_RATIO={MASK_RATIO}')
print(f'모델: {model_labels}')


NCBI email: dlalswo0321@gmail.com
ITERATIONS=30, MASK_RATIO=0.2
모델: ['DNABERT-2', 'NT-v2-50m', 'NT-v2-500m']


In [ ]:
device = get_device()
print(f'사용 중인 디바이스: {device}')


PyTorch Version: 2.9.0+cu126
Using device: cuda
사용 중인 디바이스: cuda


In [ ]:
from copy import deepcopy

gene_dict = deepcopy(DEFAULT_GENES)

print('서열 다운로드 시작...')
for gene, info in gene_dict.items():
    try:
        seq = get_sequence_from_ensembl(info['id'], seq_type='genomic')
        gene_dict[gene]['sequence'] = seq
        print(f"[{gene}] 서열 확보 완료 (길이: {len(seq)} bp)")
    except Exception as e:
        print(f"[{gene}] 에러 발생: {e}")

print('\n--- 결과 샘플 (GAPDH) ---')
print(gene_dict['GAPDH']['sequence'][:50])


서열 다운로드 시작...
[PTEN] 서열 확보 완료 (길이: 109293 bp)
[PTENP1] 서열 확보 완료 (길이: 16585 bp)
[GAPDH] 서열 확보 완료 (길이: 4919 bp)
[GAPDHP1] 서열 확보 완료 (길이: 1005 bp)
[HBB] 서열 확보 완료 (길이: 3932 bp)
[H19] 서열 확보 완료 (길이: 9388 bp)
[RPS29] 서열 확보 완료 (길이: 10008 bp)
[GAS5] 서열 확보 완료 (길이: 17517 bp)
[TP53] 서열 확보 완료 (길이: 25768 bp)
[SNHG1] 서열 확보 완료 (길이: 2044 bp)

--- 결과 샘플 (GAPDH) ---
GTCACTACCGCAGAGCCTCGAGGAGAAGTTCCCCAACTTTCCCGCCTCTC


In [ ]:
# --- 0. 긴 유전자 길이 맞춤 (Preprocessing) ---
# Colab GPU 한계(약 5~6kb) 및 비교 실험의 공정성을 위해 길이를 맞춥니다.
MAX_SEQ_LEN = 10000  # 6kb (이 정도면 안전하고 충분합니다)

print("✂️ Sequence Length Adjustment Check:")
for gene, info in gene_dict.items():
    if info['sequence'] is None:
        continue

    seq_len = len(info['sequence'])

    if seq_len > MAX_SEQ_LEN:
        # 서열이 너무 길면 앞부분만 남기고 자름
        info['sequence'] = info['sequence'][:MAX_SEQ_LEN]
        print(f"  - {gene}: {seq_len} bp -> {MAX_SEQ_LEN} bp (Cropped)")
    else:
        print(f"  - {gene}: {seq_len} bp (Keep)")

gene_selection = {
    gene: info['sequence'] for gene, info in gene_dict.items()
    if isinstance(info.get('sequence'), str) and info.get('sequence')
}


✂️ Sequence Length Adjustment Check:
  - PTEN: 109293 bp -> 10000 bp (Cropped)
  - PTENP1: 16585 bp -> 10000 bp (Cropped)
  - GAPDH: 4919 bp (Keep)
  - GAPDHP1: 1005 bp (Keep)
  - HBB: 3932 bp (Keep)
  - H19: 9388 bp (Keep)
  - RPS29: 10008 bp -> 10000 bp (Cropped)
  - GAS5: 17517 bp -> 10000 bp (Cropped)
  - TP53: 25768 bp -> 10000 bp (Cropped)
  - SNHG1: 2044 bp (Keep)


In [ ]:
import warnings
warnings.filterwarnings(
    "ignore",
    message="Increasing alibi size"
)

print('시퀀스 생성 중...')
for model_label, model_instance in iter_models(
    device, model_configs, torch_dtype=torch_dtype
):
    print(f'모델: {model_label}')

    model_name = model_label.replace('/', '-')
    model_dir = SEQ_DIR / model_name
    model_dir.mkdir(parents=True, exist_ok=True)

    gene_iter = tqdm(
        gene_selection.items(),
        desc=f'{model_label} genes',
        leave=False,
    )

    for gene_id, original_sequence in gene_iter:
        output_csv = model_dir / f'{gene_id}.csv'
        results_data = {}

        for strategy_base_key, strategy_cfg in DEFAULT_DECODING_STRATEGIES.items():
            strategy_type = strategy_cfg['type']
            temperatures = strategy_cfg.get('temperatures', [1.0])
            top_k = strategy_cfg.get('top_k', 50)

            for temp in temperatures:
                if strategy_type == 'greedy':
                    strategy_key = strategy_base_key
                else:
                    strategy_key = f'{strategy_base_key}_t{temp}'

                generated_sequences = model_instance.run(
                    sequence=original_sequence,
                    steps=ITERATIONS,
                    mask_ratio=MASK_RATIO,
                    strategy=strategy_type,
                    temperature=temp,
                    top_k=top_k,
                    save_all=True,
                    save_interval=1,
                )

                results_data[strategy_key] = generated_sequences

                del generated_sequences
                gc.collect()
                if device == 'cuda':
                    torch.cuda.empty_cache()
                elif device == 'mps':
                    torch.mps.empty_cache()

        df = pd.DataFrame(results_data).T
        df.columns = [f'iteration_{i}' for i in range(df.shape[1])]
        df.to_csv(output_csv)

    del model_instance
    gc.collect()
    if device == 'cuda':
        torch.cuda.empty_cache()
    elif device == 'mps':
        torch.mps.empty_cache()

print('✅ 시퀀스 생성 완료')

removed = 0
for cache_file in EMB_DIR.glob('**/*.pkl'):
    try:
        cache_file.unlink()
        removed += 1
    except Exception as e:
        print(f'  캐시 삭제 실패: {cache_file} ({e})')
print(f'🧹 기존 임베딩 캐시 삭제 완료: {removed} files')


시퀀스 생성 중...
📥 Downloading DNABERT-2...


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

.gitattributes: 0.00B [00:00, ?B/s]

bert_padding.py: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

modeling_esm.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/InstaDeepAI/nucleotide-transformer-v2-500m-multi-species:
- modeling_esm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

[NT-v2-500m] Model loaded successfully.
✅ NT-v2-500m loaded successfully.
모델: NT-v2-500m


✅ 시퀀스 생성 완료
🧹 기존 임베딩 캐시 삭제 완료: 0 files


In [ ]:
# 모든 시퀀스 로드
all_sequences = {}

for model_key in model_labels:
    model_name = model_key.replace('/', '-')
    model_dir = SEQ_DIR / model_name
    all_sequences[model_name] = {}

    for csv_file in model_dir.glob('*.csv'):
        gene_id = csv_file.stem
        df = pd.read_csv(csv_file, index_col=0)
        sequences_dict = {}
        for strategy in df.index:
            sequences_dict[strategy] = df.loc[strategy].tolist()
        all_sequences[model_name][gene_id] = sequences_dict

print('✅ 시퀀스 로드 완료')


✅ 시퀀스 로드 완료


In [ ]:
import warnings
warnings.filterwarnings(
    "ignore",
    message="Increasing alibi size"
)

def build_cross_embeddings(source_sequences, target_model_instance):
    cross_emb = {}
    for gene_id, strategies in source_sequences.items():
        cross_emb[gene_id] = {}
        for strategy, sequences in strategies.items():
            if not sequences:
                continue
            embeddings = []
            for seq in sequences:
                if pd.isna(seq) or seq == '':
                    continue
                emb = target_model_instance.get_embedding(str(seq))
                embeddings.append(emb)
            if embeddings:
                cross_emb[gene_id][strategy] = embeddings
            gc.collect()
            if device == 'cuda':
                torch.cuda.empty_cache()
            elif device == 'mps':
                torch.mps.empty_cache()
    return cross_emb

print('임베딩 생성 중...')
generator_labels = list(model_labels)
evaluator_labels = list(model_labels)

cross_embeddings = {}

for eval_label in evaluator_labels:
    model_instance = load_model(
        device,
        eval_label,
        model_configs[eval_label],
        torch_dtype=torch_dtype,
    )

    # if eval_label == 'DNABERT-2':
    #     _ = model_instance.get_embedding('A' * 1024)


    for gen_label in generator_labels:
        cache_name = f'embeddings_{gen_label}__by__{eval_label}.pkl'
        cache_path = EMB_DIR / cache_name

        print(f'임베딩 생성: {gen_label} sequences → {eval_label} evaluator')
        cross_emb = build_cross_embeddings(
            all_sequences[gen_label.replace('/', '-')],
            model_instance,
        )
        cross_embeddings[(gen_label, eval_label)] = cross_emb
        with open(cache_path, 'wb') as f:
            import pickle
            pickle.dump(cross_emb, f, protocol=4)
        print(f'  ✓ 저장: {cache_path}')

    del model_instance
    gc.collect()
    if device == 'cuda':
        torch.cuda.empty_cache()
    elif device == 'mps':
        torch.mps.empty_cache()

print('✅ 임베딩 생성 완료')


임베딩 생성 중...
📥 Downloading DNABERT-2...


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

✅ DNABERT-2 Triton patch applied successfully.
[DNABERT-2] Loading model...
[DNABERT-2] Model loaded successfully.
임베딩 생성: DNABERT-2 sequences → DNABERT-2 evaluator


KeyboardInterrupt: 

In [ ]:
import os
from google.colab import files

# Define the parent directory to be zipped
results_dir = "data"

# Check if the results directory exists and contains files/subdirectories
if os.path.exists(results_dir) and (os.listdir(results_dir) or os.path.isdir(results_dir)):
    print(f"Zipping '{results_dir}'...")
    !zip -r /content/data.zip {results_dir}
    print(f"✅ '{results_dir}' zipped to /content/data.zip")
    files.download('/content/data.zip')
else:
    print(f"Skipping '{results_dir}' as it is empty or does not exist.")

print("Download process initiated.")